In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from ultralytics import YOLO
from keras.models import load_model


In [5]:
model = YOLO('models/yolov8m-seg.pt')

classifier = load_model('models\\apple_classifier.h5')

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

classifier.summary()

def preprocess_for_classification(img):
    img = cv2.resize(img, (150, 150))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

def classify_apple(apple_img):
    print(f'Original apple image shape: {apple_img.shape}')
    processed_img = preprocess_for_classification(apple_img)
    print(f'Processed image shape for classification: {processed_img.shape}')
    preds = classifier.predict(processed_img)
    class_idx = np.argmax(preds)
    return class_idx

image_path = 'testimages\Apples 088.png'
image = cv2.imread(image_path)

results = model(image)

for result in results:
    boxes = result.boxes
    for box in boxes:
        class_id = int(box.cls)
        if class_id == 47:
            x1, y1, x2, y2 = box.xyxy[0].int()
            apple_img = image[int(y1):int(y2), int(x1):int(x2)]

            if apple_img.size == 0:
                continue
            
            apple_class = classify_apple(apple_img)

            class_labels = {0: "Ripe", 1: "Rotten", 2: "Unripe"}
            label = class_labels.get(apple_class, "Unknown")
            red=green=256
            if(label=="Ripe"):
                red=0
            elif(label=="Rotten"):
                green=0
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, green, red), 2)
            cv2.putText(image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, green, red), 2)

cv2.imshow('Detected Apples', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,035,203 (72.61 MB)

 Trainable params: 19,035,203 (72.61 MB)

 Non-trainable params: 0 (0.00 B)


0: 320x640 4 apples, 638.2ms
Speed: 3.0ms preprocess, 638.2ms inference, 7.6ms postprocess per image at shape (1, 3, 320, 640)
Original apple image shape: (188, 164, 3)
Processed image shape for classification: (1, 150, 150, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Original apple image shape: (165, 156, 3)
Processed image shape for classification: (1, 150, 150, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Original apple image shape: (159, 162, 3)
Processed image shape for classification: (1, 150, 150, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Original apple image shape: (156, 157, 3)
Processed image shape for classification: (1, 150, 150, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [16]:
cv2.imwrite('output_image.jpg', image)

True